# Quantum vs Classical Comparison

This notebook demonstrates the difference between quantum-enhanced and purely classical predictions.

## Objectives
1. Train both quantum and classical models
2. Compare prediction quality
3. Analyze computational performance
4. Identify cases where quantum helps

In [ ]:
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
import time
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

from src.model import QuantumFoldModel
from src.quantum_layers import HybridQuantumClassicalBlock
from src.benchmarks import ProteinStructureEvaluator

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 1. Initialize Models

Create two models: one with quantum layers and one purely classical.

In [ ]:
# Quantum-enhanced model
model_quantum = QuantumFoldModel(
    n_layers=4,
    embed_dim=64,
    n_heads=4,
    use_quantum=True,
    n_qubits=4
).to(device)

# Classical baseline
model_classical = QuantumFoldModel(
    n_layers=4,
    embed_dim=64,
    n_heads=4,
    use_quantum=False,
    n_qubits=0
).to(device)

print(f"Quantum model parameters: {sum(p.numel() for p in model_quantum.parameters()):,}")
print(f"Classical model parameters: {sum(p.numel() for p in model_classical.parameters()):,}")

## 2. Benchmark Inference Speed

Compare computational performance.

In [ ]:
# Create dummy input
batch_size = 1
seq_len = 100
embed_dim = 64

dummy_input = {
    'sequence_encoding': torch.randn(batch_size, seq_len, 20).to(device),
    'sequence_length': torch.tensor([seq_len]).to(device)
}

# Warmup
for _ in range(5):
    with torch.no_grad():
        _ = model_quantum(dummy_input)
        _ = model_classical(dummy_input)

# Benchmark quantum
n_runs = 10
times_quantum = []
model_quantum.eval()

for _ in range(n_runs):
    start = time.time()
    with torch.no_grad():
        _ = model_quantum(dummy_input)
    if device == 'cuda':
        torch.cuda.synchronize()
    times_quantum.append(time.time() - start)

# Benchmark classical
times_classical = []
model_classical.eval()

for _ in range(n_runs):
    start = time.time()
    with torch.no_grad():
        _ = model_classical(dummy_input)
    if device == 'cuda':
        torch.cuda.synchronize()
    times_classical.append(time.time() - start)

# Results
print(f"\nInference Time (100 residues, {n_runs} runs):")
print(f"  Quantum:   {np.mean(times_quantum)*1000:.1f} ± {np.std(times_quantum)*1000:.1f} ms")
print(f"  Classical: {np.mean(times_classical)*1000:.1f} ± {np.std(times_classical)*1000:.1f} ms")
print(f"  Overhead:  {np.mean(times_quantum)/np.mean(times_classical):.2f}x")

## 3. Visualize Speed Comparison

Plot inference times across different sequence lengths.

In [ ]:
# Test different sequence lengths
seq_lengths = [50, 100, 200, 300]
quantum_times = []
classical_times = []

for seq_len in seq_lengths:
    test_input = {
        'sequence_encoding': torch.randn(1, seq_len, 20).to(device),
        'sequence_length': torch.tensor([seq_len]).to(device)
    }
    
    # Quantum
    times = []
    for _ in range(5):
        start = time.time()
        with torch.no_grad():
            _ = model_quantum(test_input)
        if device == 'cuda':
            torch.cuda.synchronize()
        times.append(time.time() - start)
    quantum_times.append(np.mean(times))
    
    # Classical
    times = []
    for _ in range(5):
        start = time.time()
        with torch.no_grad():
            _ = model_classical(test_input)
        if device == 'cuda':
            torch.cuda.synchronize()
        times.append(time.time() - start)
    classical_times.append(np.mean(times))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(seq_lengths, [t*1000 for t in quantum_times], 'o-', label='Quantum', linewidth=2)
plt.plot(seq_lengths, [t*1000 for t in classical_times], 's-', label='Classical', linewidth=2)
plt.xlabel('Sequence Length')
plt.ylabel('Inference Time (ms)')
plt.title('Computational Performance: Quantum vs Classical')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Prediction Quality Comparison

Compare actual prediction accuracy on test proteins.

In [ ]:
# Generate synthetic test data
n_test = 20
test_results_quantum = []
test_results_classical = []
evaluator = ProteinStructureEvaluator()

for i in range(n_test):
    # Create synthetic data
    seq_len = np.random.randint(50, 150)
    test_input = {
        'sequence_encoding': torch.randn(1, seq_len, 20).to(device),
        'sequence_length': torch.tensor([seq_len]).to(device)
    }
    
    # Ground truth (synthetic)
    coords_true = np.random.randn(seq_len, 3) * 10
    
    # Quantum prediction
    with torch.no_grad():
        pred_quantum = model_quantum(test_input)
    coords_quantum = pred_quantum['coordinates'].cpu().numpy()[0]
    
    # Classical prediction
    with torch.no_grad():
        pred_classical = model_classical(test_input)
    coords_classical = pred_classical['coordinates'].cpu().numpy()[0]
    
    # Evaluate
    metrics_q = evaluator.evaluate_structure(coords_quantum, coords_true, seq_len)
    metrics_c = evaluator.evaluate_structure(coords_classical, coords_true, seq_len)
    
    test_results_quantum.append(metrics_q.to_dict())
    test_results_classical.append(metrics_c.to_dict())

print(f"Evaluated {n_test} test proteins")

## 5. Statistical Analysis

In [ ]:
# Extract metrics
metrics_names = ['rmsd', 'tm_score', 'gdt_ts', 'lddt']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, metric in enumerate(metrics_names):
    values_q = [r[metric] for r in test_results_quantum]
    values_c = [r[metric] for r in test_results_classical]
    
    ax = axes[idx]
    
    positions = [1, 2]
    data = [values_c, values_q]
    
    bp = ax.boxplot(data, positions=positions, widths=0.6, patch_artist=True,
                     labels=['Classical', 'Quantum'])
    
    # Color boxes
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][1].set_facecolor('lightcoral')
    
    ax.set_ylabel(metric.upper())
    ax.set_title(f'{metric.upper()} Comparison')
    ax.grid(axis='y', alpha=0.3)
    
    # Add mean values
    mean_c = np.mean(values_c)
    mean_q = np.mean(values_q)
    ax.text(1, mean_c, f'{mean_c:.3f}', ha='center', va='bottom')
    ax.text(2, mean_q, f'{mean_q:.3f}', ha='center', va='bottom')
    
    # Statistical test
    from scipy import stats
    t_stat, p_value = stats.ttest_ind(values_q, values_c)
    ax.text(1.5, ax.get_ylim()[1]*0.95, f'p={p_value:.4f}', ha='center')

plt.tight_layout()
plt.show()

# Print summary
print("\nSummary Statistics:")
for metric in metrics_names:
    values_q = [r[metric] for r in test_results_quantum]
    values_c = [r[metric] for r in test_results_classical]
    
    print(f"\n{metric.upper()}:")
    print(f"  Classical: {np.mean(values_c):.4f} ± {np.std(values_c):.4f}")
    print(f"  Quantum:   {np.mean(values_q):.4f} ± {np.std(values_q):.4f}")
    improvement = ((np.mean(values_q) - np.mean(values_c)) / np.mean(values_c)) * 100
    print(f"  Improvement: {improvement:+.2f}%")

## Conclusions

From this analysis, we can observe:

1. **Computational Cost**: Quantum layers add overhead due to circuit simulation
2. **Prediction Quality**: Results will vary based on the specific test case
3. **Trade-offs**: Consider speed vs accuracy for your use case

Key insights:
- Quantum advantage may emerge for specific protein classes
- Hardware quantum processors could reduce computational overhead
- Hybrid approach allows flexibility based on requirements